In [ ]:
import sys
sys.path.append("..")
from Functions.UNSW_DF import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train, test = train_test_import()

X = train.drop(["label"], axis=1)
y = train["label"]

cols = X.columns
for df in[X]:
    for col in cols:
        df[col] = df[col].astype(float)
X.info()

## Algorithm (Pseudo-code)

In [ ]:
# TODO: (1) Define a function to calculate the distance between two points
# TODO: (2) Use the distance function to get the distance between a test point and all known data points
# TODO: (3) Sort distance measurements to find the points closest to the test point (i.e., find the nearest neighbors)
# TODO: (4) Use majority class labels of those closest points to predict the label of the test point
# TODO: (5) Repeat steps 1 through 4 until all test data points are classified

In [ ]:
# ATTEMPT
#import numpy as np
#def euclidean_distance(a, b):
#    dim = len(a)
#    dist = 0
#    
#    for i in range(dim):
#        dist = dist.append(np.linalg.norm(a-b))
#    return dist

## (1) Define a function to calculate distance between two points

In [ ]:
# Euclidean Distance
#def euclidean_distance(a, b):
#    #compute distances between all points
#    distances = np.zeros((len(b),len(a)))
#    for i in range(len(b)):
#        for j in range(len(a)):
#            distances[i,j] = (a[j] - b[i])**2
#    return distances
#dist = euclidean_distance(a=X.iloc[0], b=X.iloc[1])


In [64]:
def euclidean_distance(a, b, p=2):
    # Store the number of dimensions
    dim = len(a)
    # Set initial distance to 0
    distance = 0
    for i in range(dim):
        dist = np.linalg.norm(a-b)
        dist = dist**(1/p)
    return dist
list = euclidean_distance(X.iloc[0], X.iloc[1])

## 2. Use the distance function to get distance between a test point and all known data points

In [65]:
# Define an arbitrary test point
test_pt = [4.8, 2.7, 2.5, 0.7]

# Calculate distance between test_pt and all points in X
distances = []

for i in X.index:
    distances.append(euclidean_distance(test_pt, X.iloc[i]))
    
df_dists = pd.DataFrame(data=distances, index=X.index, columns=['dist'])
df_dists.head()

ValueError: operands could not be broadcast together with shapes (4,) (53,) 

## 3. Sort distance measurements to find the points closest to the test point

In [ ]:
# Find the 5 nearest neighbors
df_nn = df_dists.sort_values(by=['dist'], axis=0)[:5]
df_nn

## 4. Use majority class labels of those closest points to predict the label of the test point

In [ ]:
from collections import Counter

# Create counter object to track the labels
counter = Counter(y[df_nn.index])

# Get most common label of all the nearest neighbors
counter.most_common()[0][0]

## 5. Repeat steps 1 through 4 until all test data points are classified

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the data - 75% train, 25% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

# Scale the X data
#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)
X_train.info()

In [ ]:
def knn_predict(X_train, X_test, y_train, y_test, k, p):
    
    # Counter to help with label voting
    from collections import Counter
    
    # Make predictions on the test data
    # Need output of 1 prediction per test data point
    y_hat_test = []

    for test_point in X_test:
        distances = []

        for train_point in X_train:
            distance = minkowski_distance(test_point, train_point, p=p)
            distances.append(distance)
        
        # Store distances in a dataframe
        df_dists = pd.DataFrame(data=distances, columns=['dist'], 
                                index=y_train.index)
        
        # Sort distances, and only consider the k closest points
        df_nn = df_dists.sort_values(by=['dist'], axis=0)[:k]

        # Create counter object to track the labels of k closest neighbors
        counter = Counter(y_train[df_nn.index])

        # Get most common label of all the nearest neighbors
        prediction = counter.most_common()[0][0]
        
        # Append prediction to output list
        y_hat_test.append(prediction)
        
    return y_hat_test

In [ ]:
# Make predictions on test dataset
y_hat_test = knn_predict(X, X_test, y, y_test, k=5, p=1)

print(y_hat_test)